# Appmode in Jupyter

This page demonstrates how to generate online "apps" with a Jupyter Notebook interface. With the `appmode` plugin, we can create interactive experiences without requiring coding or running a specific cell.

Markdown cells will be retained, and all code cells will be run, then hidden. The outputs of each cell will be displayed as well.

Check out a further example using an [ipyvolume](https://ipyvolume.readthedocs.io/en/latest/) to render a 3D plot: [ipyvolume_demo.ipynb](ipyvolume_demo.ipynb)

In [ ]:
#!pip install pandas

In [1]:
from __future__ import print_function #
import IPython
from IPython.display import Javascript,clear_output,display #
from ipywidgets import interact, interactive, fixed, interact_manual,Layout,jslink,Button, Layout, jslink, IntText, IntSlider,TwoByTwoLayout #
import ipywidgets as widgets
import copy #
from bs4 import BeautifulSoup 
from urllib.request import Request, urlopen #
import pandas as pd #
import requests #
from io import StringIO #
from pychord import Chord

In [2]:
# import song list
def song_table():
    orig_url='https://drive.google.com/file/d/1aDrlFu5eSabjo8CVKAOG6ljfr5OihZLq/view?usp=sharing'
    file_id = orig_url.split('/')[-2]
    dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
    text = requests.get(dwn_url)
    text.encoding = 'utf-8'
    csv_raw = StringIO(text.text)
    df = pd.read_csv(csv_raw,encoding='utf-8')
    df['אמן'] = df['אמן'].apply(lambda x: x.strip())
    df['שיר'] = df['שיר'].apply(lambda x: x.strip())
    df['shir_aman'] = df['אמן']+" | "+df['שיר'] 
    return df

df = song_table()

In [3]:
def artist_songs(artist_name='א'):
      song_list = df.loc[df['shir_aman'].str.contains(artist_name)]['shir_aman'].sort_values()
      song_list = list(zip(song_list,song_list.index))
      return song_list

In [4]:
from __future__ import division
from ipywidgets import IntSlider, Label,Text
import ipywidgets as widgets


song_tab = widgets.Select(options=artist_songs(),rows=10)


text = widgets.Text(
    value='',
    placeholder='חפש שיר או אמן',
    disabled=False
)



def song_search_change(change):
    #new_options = []
    #for i in orig_options:
        #print(i)
        #if change.new in i:
          #  new_options.append(i)
    new_options = artist_songs(change.new)
    song_tab.options = new_options
   # drop.value = change.new 

text.observe(song_search_change, names='value')
display(text,song_tab)

Text(value='', placeholder='חפש שיר או אמן')

Select(options=(('Adi Ulmansky | איך לאהוב אותי', 8703), ('Chesher | לפרק את מדינת תל אביב', 5360), ('Eitan Ka…

In [5]:
song_tab.value

8703

In [6]:
from IPython.display import display, Javascript

disable_js = """
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
"""

In [7]:

from IPython.display import display,clear_output
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
      clear_output()
      #with output:
      #display(IPython.display.HTML("<marquee style='width: 30%; color: purple;'><b>✌ ...טוען שיר</b></marquee>"))
      #clear_output()
      myurl = df.iloc[song_tab.value]['לינק']
      req = Request(myurl, headers={'User-Agent': 'Mozilla/5.0'})
      webpage = urlopen(req).read()
      soupified = BeautifulSoup(webpage,'lxml' ) # 'html.parser'
      song = soupified.find("div", {"class": "song_block"}) #Find the nevessary tag and class which it belongs to
      header = song.find(attrs={"class":"song_block_title"})
      # Clean the HTML from ads and more unnecessary stuff
      header = header.get_text().strip().replace("אקורדים לשיר ","")

      text_alignment = 'right'
      header = f"""<h1 style="text-align: {text_alignment}">"""+header+'</h1>'
      on_button_clicked.header = header

      progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
      header_progression = header+progression_status

      delete_nodes = [{"class":"song_rating"},
                      {"class":"row"},
                      {"id":"songad"},
                      {"class":"song_block_title"},
                      {"id":"songTopAd8Td"},
                      {"class":"videosTitle"},
                      {"type":"text/javascript"},
                      {"id":"taboola-below-article-thumbnails"},
                      {"id":"downInSongTable"},
                      {"class":"chord_info"}]

      for node in delete_nodes:
        for div in song.find_all(attrs=node): 
            div.decompose()

      # Align guitar TABS to the left
      tab_list = ''
      tab_nodes = [{"class":"tabs"}]
      for node in tab_nodes:
        for div in song.find_all(attrs=node):
          div['dir'] = 'ltr'
          div['align'] = 'left'
          #div['display'] = 'inline-block'
          #div['width'] = '300px'
          div['position'] = 'absolute'
          div['right'] = '10%'
          #div['margin-left'] = '300px'
          tab_list=tab_list+'<p'+str(div)+'</p>'
      on_button_clicked.song=song
      # Merge Header & Song(words+chords)
      full_page=header_progression+str(song)
      on_button_clicked.full_page = full_page
      display(IPython.display.HTML(full_page))

display(Javascript(disable_js))
button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()

<IPython.core.display.Javascript object>